In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from typing import List
import numpy as np
import joblib
from pprint import pprint
import imodelsx.util
import sasc.viz
import pickle as pkl
import json
from copy import deepcopy
from numpy.linalg import norm
from sasc.config import CACHE_DIR, RESULTS_DIR, cache_ngrams_dir, regions_idxs_dir
import sasc.modules.fmri_module
ngrams_list = joblib.load(join(cache_ngrams_dir, 'fmri_UTS02_ngrams.pkl')) # ngrams are same for both models

# subject = 'S02'
subject = 'S03'
rois_dict = joblib.load(join(regions_idxs_dir, f'rois_{subject}.jbl'))

### Get predictions from embs

In [ ]:
embs = joblib.load(join(cache_ngrams_dir, 'fmri_embs.pkl'))
# embs = joblib.load(join(cache_ngrams_dir, 'fmri_embs_llama.pkl'))
mod = sasc.modules.fmri_module.fMRIModule(
    subject=f"UT{subject}",
    checkpoint="facebook/opt-30b",
    # checkpoint="huggyllama/llama-30b",
    init_model=False,
    restrict_weights=False,
)
voxel_preds = mod(embs=embs, return_all=True)
outputs_dict = {
    k: voxel_preds[:, np.array(rois_dict[k])].mean(axis=1)
    for k in rois_dict
}
joblib.dump(outputs_dict, join(
    cache_ngrams_dir, f'rois_ngram_outputs_dict_{subject}_opt.pkl'))
# cache_ngrams_dir, f'rois_ngram_outputs_dict_{subject}_llama.pkl'))

In [ ]:
outputs_dict = joblib.load(
    join(cache_ngrams_dir, f'rois_ngram_outputs_dict_{subject}_opt.pkl'))
df_opt = pd.DataFrame(outputs_dict, index=ngrams_list)
outputs_dict = joblib.load(
    join(cache_ngrams_dir, f'rois_ngram_outputs_dict_{subject}_llama.pkl'))
df_llama = pd.DataFrame(outputs_dict, index=ngrams_list)
df = df_opt + df_llama
ROIS_LOC = ['RSC', 'PPA']  # 'OPA', 'PPA']
for k in ROIS_LOC:
    df_opt[k + '_only'] = df_opt[k] - \
        df_opt[[c for c in ROIS_LOC if c != k]].mean(axis=1)
    df_llama[k + '_only'] = df_llama[k] - \
        df_llama[[c for c in ROIS_LOC if c != k]].mean(axis=1)
    df[k + '_only'] = df[k] - \
        df[[c for c in ROIS_LOC if c != k]].mean(axis=1)

In [ ]:
stability_scores = {
    k: np.corrcoef(df_opt[k], df_llama[k])[0, 1]
    for k in df.columns
}

In [ ]:
ascending = False  # should be false to get driving ngrams
top_ngrams_dict = {}
for k in df.columns:
    top_ngrams_dict[k] = df.sort_values(
        k, ascending=ascending).index[:100].tolist()
    # if k in ROIS_LOC:

    # top_ngrams_dict[k + '_only'] = df.sort_values(
    # k + '_only', ascending=ascending).index[:100].tolist()
top_ngrams_df = pd.DataFrame(top_ngrams_dict)
top_ngrams_df.to_csv(f'top_ngrams_by_roi_{subject}.csv')
with pd.option_context('display.max_rows', None):
    # rois = ['RSC', 'OPA', 'PPA', 'IPS', 'pSTS', 'sPMv',
    # 'EBA', 'OFA'] + ['RSC_only', 'OPA_only', 'PPA_only']
    rois = ['RSC', 'PPA', 'IPS', 'EBA', 'OFA']  # + ['RSC_only', 'PPA_only']
    display(top_ngrams_df[rois])

In [ ]:
gpt4 = imodelsx.llm.get_llm('gpt-4-turbo-0125-spot')
try:
    explanations = json.load(open(f'explanations_by_roi_{subject}.json', 'r'))
except:
    explanations = {}
for k in top_ngrams_df.columns:

    s = '- ' + '\n- '.join(top_ngrams_df[k].iloc[:60])
    prompt = f'''Here is a list of phrases:
    {s}

    What is a common theme among these phrases? Return only a concise phrase.'''
    if not k in explanations:
        explanations[k] = gpt4(prompt)
json.dump(explanations, open(
    f'explanations_by_roi_{subject}.json', 'w'), indent=4)

In [21]:
{k: explanations[k] for k in rois}

{'RSC': 'Movement and locations',
 'PPA': 'Locations and settings',
 'IPS': 'Spatial positioning and movement',
 'EBA': 'Physical contact and movement',
 'OFA': 'Conversational transitions'}

### S03 Export selected rois to pkl

In [27]:
# rois = ['RSC', 'OPA', 'PPA', 'IPS', 'pSTS', 'sPMv',
# 'EBA', 'OFA'] + ['RSC_only', 'OPA_only', 'PPA_only2']  # 'PPA_only1',
rois = ['RSC', 'PPA', 'IPS', 'EBA', 'OFA']  # + ['RSC_only', 'PPA_only']
# pprint({k: explanations[k] for k in rois})
explanations_clean = {
    'EBA': 'Body parts',
    'IPS': 'Descriptive elements of scenes or objects',
    'OFA': 'Conversational transitions',  # THIS DIFFERS FROM UTS02
    # 'OPA': 'Direction and location descriptions',
    # 'OPA_only': 'Spatial positioning and directions',
    'PPA': 'Scenes and settings',
    # 'PPA_only': 'Lying and falsehoods',
    'RSC': 'Travel and location names',
    # 'RSC_only': 'Location names',
    # 'pSTS': 'Verbal interactions',
    # 'sPMv': 'Time and numbers'
}
explanation_avoid_suffixes = {
    'EBA': ' Avoid mentioning any locations.',
    'IPS': ' Avoid mentioning any locations.',
    'OFA': ' Avoid mentioning any locations.',
    # 'OPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    # 'OPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'PPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    # 'PPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'RSC': '',
    # 'RSC_only': '',
    # 'pSTS': ' Avoid mentioning any locations.',
    # 'sPMv': ' Avoid mentioning any locations.'
}
for roi in rois:
    print(f'"{roi}":', str(
        top_ngrams_df[roi.replace('1', '').replace('2', '')].iloc[:50].values.tolist()) + ', ')
top_ngrams_clean = {
    "RSC": ['was led upstairs', 'onto the subway', 'to the hallway', 'drove to washington', 'back through london', 'and darted downstairs', 'past the offices', 'long hallway toward', 'down the sidewalk', 'back in manhattan', 'reached the interstate', 'just blocks away', 'drove from vermont', 'was standing outside', 'to a courtyard', 'in the alley', 'up the coast', 'from my dorm', 'in the courtyard', 'in central park', 'i walk outside', 'here in manhattan', 'darted downstairs', 'facing the beach', 'walk through downtown', 'wander the hallways', 'i ran downstairs', 'down the hall', "'m standing outside", 'off into vancouver', 'through the streets', 'sitting in indianapolis', 'on sixth avenue', 'i go upstairs', 'across the street', 'arrived in indianapolis', 'we were downtown'],
    # "OPA": [],
    "PPA": ['in an alcove', 'on the stoop', 'past the offices', 'against the railing', 'on the windowsill', 'in the alley', 'to a courtyard', 'the copier room', 'in the courtyard', 'this long hallway', 'to the hallway', 'on a dock', 'in the hallway', 'long hallway toward', 'outside the windows', 'on that terrace', 'inside the hut', 'railing looking out', 'through the windows', 'down this embankment', 'on the subway', 'onto the subway', 'there were shelves', 'in my cubicle', 'a strip mall', 'on the sidewalk', 'in the colonnade', 'on the railing', 'into the basement', 'across the parking', 'a restaurant stoop', 'onto the railing', 'exit the subway', 'by the window', 'in that attic', 'was led upstairs', 'in the basement', 'the food court', 'and the courtyard', 'in the cafeteria', 'hall past the', 'into the parking', 'in the windowless', 'back room where', 'on my bed', 'down the sidewalk', 'contain strip malls', 'onto the sidewalk', 'the hall closet', 'at those cliffs'],


    # "RSC_only": [],
    # "OPA_only": [],
    # "PPA_only": [],

    "IPS": ['and behind me', 'against the railing', 'onto the railing', 'path that jutted', 'situated herself behind', 'above the gulf', 'door behind me', 'southern shore of', 'i looked across', 'along the edge', 'closed behind me', 'behind me and', 'across a plateau', 'on the railing', 'up behind me', 'leaning against the', 'towering above me', 'jutted into the', 'onto the bridge', 'and cut across', 'behind him and', 'and came around', 'front of us', 'up onto the', 'over the gulf', 'stood behind me', 'across the bridge', 'beside the river'],
    # "pSTS": [],
    # "sPMv": [],
    "EBA": ['arms around her', 'wraps his arms', 'hands gripped the', 'into my palm', 'hands into my', 'elbows on knees', 'grab his arms', 'his hands folded', 'into her arms', 'grabbed her hand', 'arms flailing', 'grabbed her legs', 'arm around my', 'grabbed their hands', 'lifted her up', 'put my arms', 'leaned his head', 'put his arms', 'shakes my hand', 'flying arms flailing', 'i leaned down', 'arms tighten around', 'her hands gripped', 'hand on his', 'my feet kicking', 'pinning my arms', 'held her hand', 'in a headlock', 'pressed my face', 'holds her hand', 'arms flailing holding', 'rubbing his head'],
    "OFA": ['and we talked', 'i even met', 'and so finally', 'one night my', 'one evening after', 'anyway the point', 'one summer my', 'weeks passed and', 'finally one day', 'then we talked', 'one night i', 'we chatted', 'and i talked', 'we talked and', 'talked and', 'so i texted', 'to my surprise', 'one afternoon when', 'i persisted and', 'and i finally', 'was watching television', 'i remember once', 'so one night', 'but anyway', 'and i met', 'when i finally', 'so i finally', 'on and on', 'son had finally', 'and i especially', 'so one day', 'and eventually i', 'we brace ourselves', 'so anyway', 'i was perusing', 'and it finally', 'later that day', 'so we finally', 'but anyhow', 'and as we', 'the day came', 'home one afternoon', 'then i finally', 'what fascinated me', 'and i vaguely', 'so i talked', 'once while i', 'was hanging out', 'i was reliving', 'but the most'],
}

rows = {
    'roi': rois,
    'expl': [explanations_clean[k] for k in rois],
    'top_ngrams_module_correct': [top_ngrams_clean[k] for k in rois],
    'stability_score': [stability_scores[k.split('_')[0]] for k in rois],
    # 'question': questions,
    'subject': [f'UT{subject}'] * len(rois),
    'voxel_nums': [rois_dict[k.split('_')[0]] for k in rois],
    'prompt_suffix': [explanation_avoid_suffixes[k] for k in rois],
}
rows = pd.DataFrame(rows)
rows.to_pickle(f'rows_roi_{subject}_may31.pkl')

"RSC": ['was led upstairs', 'onto the subway', 'to the hallway', 'in the hallway', 'we ran upstairs', 'drove to washington', 'exit the subway', 'back through london', 'on the subway', 'and darted downstairs', 'past the offices', 'long hallway toward', 'down the sidewalk', 'back in manhattan', 'we go downstairs', 'reached the interstate', 'onto the sidewalk', 'this long hallway', 'just blocks away', 'up the stairs', 'to my dorm', 'in lower manhattan', 'on the sidewalk', 'down the corridor', 'took the subway', 'drove from vermont', 'was standing outside', 'to a courtyard', 'in the alley', 'up the coast', 'from my dorm', 'in the courtyard', 'in central park', 'i walk outside', 'here in manhattan', 'darted downstairs', 'facing the beach', 'walk through downtown', 'wander the hallways', 'i ran downstairs', 'down the hall', "'m standing outside", 'off into vancouver', 'through the streets', 'sitting in indianapolis', 'on sixth avenue', 'i go upstairs', 'across the street', 'arrived in indian

### S02 Export selected rois to pkl

In [ ]:
rois = ['RSC', 'OPA', 'PPA', 'IPS', 'pSTS', 'sPMv',
        'EBA', 'OFA'] + ['RSC_only', 'OPA_only', 'PPA_only2']  # 'PPA_only1',
# pprint({k: explanations[k] for k in rois})
explanations_clean = {
    'EBA': 'Body parts',
    'IPS': 'Descriptive elements of scenes or objects',
    'OFA': 'Personal growth and reflection',
    'OPA': 'Direction and location descriptions',
    'OPA_only': 'Spatial positioning and directions',
    'PPA': 'Scenes and settings',
    'PPA_only': 'Unappetizing foods',
    'RSC': 'Travel and location names',
    'RSC_only': 'Location names',
    'pSTS': 'Verbal interactions',
    'sPMv': 'Time and numbers'}
explanation_avoid_suffixes = {
    'EBA': ' Avoid mentioning any locations.',
    'IPS': ' Avoid mentioning any locations.',
    'OFA': ' Avoid mentioning any locations.',
    'OPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'OPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'PPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'PPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'RSC': '',
    'RSC_only': '',
    'pSTS': ' Avoid mentioning any locations.',
    'sPMv': ' Avoid mentioning any locations.'
}
for roi in rois:
    print(f'"{roi}":', str(
        top_ngrams_df[roi.replace('1', '').replace('2', '')].iloc[:50].values.tolist()) + ', ')
    # {
    # roi:  for roi in rois
# })
top_ngrams_clean = {
    "RSC": ['drove from vermont', 'to washington', 'in manhattan', 'here in boston', 'off into vancouver', 'moved to chicago', 'was in mexico', 'arrived in indianapolis', 'came to florida', 'i left vermont'],
    "OPA": ['onto the railing', 'towards the river', 'onto the sidewalk', 'towards the doors', 'outside the windows', 'long hallway toward', 'to the horizon', 'towards the street', 'over the gulf', 'to my left', 'path that jutted', 'on the ceiling', 'on the windowsill', 'down this embankment', 'up those stairs', 'above the gulf', 'facing the beach'],
    "PPA": ['mile of cornfields', 'the windowsill', 'the rolling hills', 'beautiful moonlit mountains', 'giant stone cliffs', 'a strip mall', 'nondescript office buildings', 'manicured lawns', 'lakes', 'the dark driveway', 'and shimmering skyscrapers', 'a private beach', 'the leafy garden', 'our modest backyard', 'my dorm'],

    "RSC_only": ['florida', 'israel', 'london', 'marrakesh', 'indianapolis', 'paris', 'pennsylvania', 'tokyo', 'tenessee', 'boston', 'vermont', 'chicago', 'indianapolis'],
    "OPA_only": ['towards the ceiling', 'onto the railing', 'feet hanging over', 'towards the doors', 'seats behind', 'towards the door', 'lights peeking over', 'to my left', 'situated herself behind', 'you sit backward', 'to the horizon', 'maybe twelve feet', 'at the ceiling', 'towards the street', 'of seats behind', 'twenty feet above', 'his back turned', 'see the horizon', 'seats behind the', 'to my right', 'and high rafters', 'about twenty feet', 'door behind me', 'the door behind', 'toward the back', 'over his shoulder', 'feet above the', 'hands went underneath', 'towards the ground', 'his feet hanging', 'feet touch the', 'behind her and', 'stand in front', 'down one side', 'on opposite sides', 'over the ceiling', 'on either side'],
    # "PPA_only": ['kind of corny', 'his painting sucked', 'snake oil', 'liar fake', 'fake name', 'bad puns', 'as an insult', 'called baloney'],
    "PPA_only2": ['like burnt steak', 'like pudding', 'tasted pretty bad', 'stale baked goods', 'the crusts', 'baloney', 'yeast extract', 'a sandwich rejected',],

    "IPS": ['there were slats', 'four connected squares', 'in long rows', 'on the sides', 'a long narrow', 'that forms horizontal', 'long rows of', 'sixty foot wide', 'between buttered slices', 'mile thick ice', 'all four corners', 'along the top'],
    "pSTS": ['said excuse me', 'says excuse me', 'room went silent', 'someone shouted', 'i provoked gasps', 'somebody then yelled', 'she started laughing', 'excuse me', 'asked i laughed', 'exhalation someone shouted', 'retorted rather loudly', 'turned and said', 'hurry she exclaimed', 'i started yelling', 'say excuse me', 'i started laughing', 'interrupted the conversation', 'breath he yelled', 'moment she gasped', 'said guess what'],
    "sPMv": ['one', 'forty', 'april nineteen forty', 'was sixteen seventeen', 'five only twenty', 'three down', 'march twentieth nineteen', 'more time passed', 'fifteen meters fifty', "turning ninety", 'june of nineteen'],
    "EBA": ['wraps his arms', 'lifted her dress', 'arms flailing', 'hands gripped the', 'grabbed her legs', 'his hands folded', 'my feet kicking', 'navigated pushy elbows', 'elbows on knees', 'over his shoulder'],
    "OFA": ['of my childhood', 'newfound self esteem', 'so my shrink', 'hurtful first dates', 'recall many instances', 'it felt magical', 'answered many questions', 'my school days', 'no satisfying fantasies', 'my mom often', 'from our childhood', 'growing up we', 'good friends often', 'shaped their mind', 'everything my parents'],
}

rows = {
    'roi': rois,
    'expl': [explanations_clean[k] for k in rois],
    'top_ngrams_module_correct': [top_ngrams_clean[k] for k in rois],
    'stability_score': [stability_scores[k.split('_')[0]] for k in rois],
    # 'question': questions,
    'subject': [f'UT{subject}'] * len(rois),
    'voxel_nums': [rois_dict[k.split('_')[0]] for k in rois],
    'prompt_suffix': [explanation_avoid_suffixes[k] for k in rois],
}
rows = pd.DataFrame(rows)
rows.to_pickle(f'rows_roi_{subject}_may31.pkl')